In [1]:
!pip install rouge-score
!pip install contractions
!pip install datasets
!pip install rouge_score
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=e25a1edc8f1f54917620714cfc015298a6c0233dbe598d5e5d885e17d38b7a5e
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge-score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 7.2 MB/s 
     |████████████████████████████████| 287 kB 15.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 6.8 MB/s 
     |████████████████████████████████| 182 kB 64.9 MB/s 
     |████████████████████████████████| 212 kB 63.8 MB/s 
     |████████████████████████████████| 132 kB 60.0 MB/s 
     |████████████████████████████████| 127 kB 69.0 MB/s 
 

In [2]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import numpy as np
nltk.download('punkt')
import re
from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer
import string
import rouge_score
import contractions

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def dataset_processing(data):
  data.drop_duplicates(inplace=True)
  data.dropna(inplace=True)
  data=data.reset_index(drop=True)
  return data

In [5]:
data=pd.read_csv("/content/drive/MyDrive/ML Assignment/train.csv",sep=",",encoding='utf-8')
data=dataset_processing(data)
INPUT_DATA=data['text']
OUTPUT_DATA=data['title']

In [6]:
# Preprocessing Configuration
punctuation=string.punctuation.replace('.',"")
mapper=contractions.contractions_dict

In [7]:
def remove_url(corpus):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',corpus)

def remove_mention(corpus):
    url = re.compile(r'@\S*')
    return url.sub(r'',corpus)

def remove_punctuation(corpus):
    table=str.maketrans('','',punctuation)
    return corpus.translate(table)

def unicode_to_ascii(corpus):
    corpus = corpus.encode('utf8').decode('ascii', 'ignore')
    return corpus

def replace_mapping(text):
    for p in mapper:
        text = text.replace(p, mapper[p])
    return text


In [8]:
def corpus_preprocessing(corpus,use_as=None):
  if use_as=="input":
    corpus="".join(corpus.split("-")[1:])

  corpus=remove_url(corpus)
  corpus=remove_mention(corpus)
  corpus=remove_punctuation(corpus)
  corpus=unicode_to_ascii(corpus)
  corpus=replace_mapping(corpus)
  return " ".join(word_tokenize(corpus.lower()))

PROCESSED_INPUT=INPUT_DATA.apply(corpus_preprocessing,use_as="input")
PROCESSED_OUTPUT=OUTPUT_DATA.apply(corpus_preprocessing)


In [9]:
final_data=pd.DataFrame({"text":PROCESSED_INPUT,"title":PROCESSED_OUTPUT})
final_data.to_csv("/content/drive/MyDrive/ML Assignment/processed_train.csv",sep=",")

In [33]:
from datasets import load_dataset,load_from_disk,load_metric
def data_preparation():
    full_dataset = load_dataset("csv", data_files="/content/drive/MyDrive/ML Assignment/processed_train.csv")
    cols_to_remove = list(full_dataset['train'].features.keys())
    cols_to_remove.remove("title")
    cols_to_remove.remove("text")
    data=full_dataset.remove_columns(cols_to_remove)
    dataset = data['train'].train_test_split(test_size=0.1)
    test_val = dataset['test'].train_test_split(test_size=0.5)
    dataset['val'] = test_val['train']
    dataset['test'] = test_val['test']
    dataset.save_to_disk("/content/drive/MyDrive/ML Assignment/final_data")
    dataset = load_from_disk("/content/drive/MyDrive/ML Assignment/final_data")
    nltk.download('punkt')
    return dataset

data=data_preparation()

  0%|          | 0/1 [00:00<?, ?it/s]

Flattening the indices:   0%|          | 0/14 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration, T5TokenizerFast
tokenizer = T5TokenizerFast.from_pretrained("t5-base",model_max_length=600)

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_

In [38]:
MAX_TEXT_LEN = 600
MAX_TITLE_LEN = 30
def preprocessing(data):
  model_inputs = tokenizer(data['text'], max_length=MAX_TEXT_LEN, padding=True, truncation=True)
  with tokenizer.as_target_tokenizer():
      labels = tokenizer(data['title'], max_length=MAX_TITLE_LEN, padding=True, truncation=True)
  labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]]
  model_inputs['labels'] = labels["input_ids"]
  return model_inputs

In [39]:
data_processed=data.map(preprocessing,batched=True,remove_columns=['text', 'title'],)

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

**MODEL DEVELOPMENT**

In [40]:
batch_size = 5
num_epochs = 2
learning_rate = 5.6e-5
weight_decay = 0.01
log_every = 50
eval_every = 500
lr_scheduler_type = "linear"

In [41]:
from transformers import TFTrainer, TFTrainingArguments, Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/ML Assignment/model-t5-base",
    evaluation_strategy="steps",
    eval_steps=eval_every,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_steps=500,
    save_total_limit=2,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    logging_steps=log_every,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    resume_from_checkpoint=True,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
from transformers import AutoModelForSeq2SeqLM,Seq2SeqTrainer,DataCollatorForSeq2Seq
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_

In [45]:
collator=DataCollatorForSeq2Seq(tokenizer,model=model)

metric = load_metric("rouge")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [46]:
trainer=Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=data_processed['train'],
    eval_dataset=data_processed['val'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13402
  Num Epochs = 2
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 5362
  Number of trainable parameters = 222903552


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
500,1.742300,1.533997,44.789900,19.711500,40.125500,40.054000
1000,1.651500,1.495683,45.689100,20.572400,41.235500,41.223100
1500,1.589000,1.451320,47.241300,22.049400,42.491400,42.428300
2000,1.562900,1.424777,46.970200,21.667900,42.464400,42.382100
2500,1.598700,1.407954,47.322500,22.013500,42.592000,42.522100
3000,1.386400,1.409599,47.372900,22.716700,42.992900,42.966100
3500,1.308800,1.402848,47.805000,22.599200,43.184800,43.142600
4000,1.308600,1.391574,47.491800,22.643700,42.881000,42.845800
4500,1.392300,1.390984,47.801100,22.773400,43.246000,43.238600
5000,1.383000,1.389388,48.056900,22.960200,43.645600,43.597300


***** Running Evaluation *****
  Num examples = 745
  Batch size = 5
Saving model checkpoint to /content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-500
Configuration saved in /content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-500/special_tokens_map.json
Copy vocab file to /content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-500/spiece.model
***** Running Evaluation *****
  Num examples = 745
  Batch size = 5
Saving model checkpoint to /content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-1000
Configuration saved in /content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-1000/config.json
Model weights saved in /conte

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
500,1.742300,1.533997,44.789900,19.711500,40.125500,40.054000
1000,1.651500,1.495683,45.689100,20.572400,41.235500,41.223100
1500,1.589000,1.451320,47.241300,22.049400,42.491400,42.428300
2000,1.562900,1.424777,46.970200,21.667900,42.464400,42.382100
2500,1.598700,1.407954,47.322500,22.013500,42.592000,42.522100
3000,1.386400,1.409599,47.372900,22.716700,42.992900,42.966100
3500,1.308800,1.402848,47.805000,22.599200,43.184800,43.142600
4000,1.308600,1.391574,47.491800,22.643700,42.881000,42.845800
4500,1.392300,1.390984,47.801100,22.773400,43.246000,43.238600
5000,1.383000,1.389388,48.056900,22.960200,43.645600,43.597300




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5362, training_loss=1.481329763882732, metrics={'train_runtime': 6091.8241, 'train_samples_per_second': 4.4, 'train_steps_per_second': 0.88, 'total_flos': 1.9127938240512e+16, 'train_loss': 1.481329763882732, 'epoch': 2.0})

In [47]:
!zip -r "/content/drive/MyDrive/ML Assignment/checkpoint-5000.zip" "/content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000"

  adding: content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000/ (stored 0%)
  adding: content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000/config.json (deflated 62%)
  adding: content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000/pytorch_model.bin (deflated 8%)
  adding: content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000/tokenizer_config.json (deflated 83%)
  adding: content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000/special_tokens_map.json (deflated 86%)
  adding: content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000/spiece.model (deflated 48%)
  adding: content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000/tokenizer.json (deflated 74%)
  adding: content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000/training_args.bin (deflated 48%)
  adding: content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000/optimizer.pt (deflated 8%)
  adding: content/drive/MyDrive/ML Assignment/mode

In [48]:
MAX_TEXT_LENGTH = 600
MAX_TITLE_LENGTH = 30

BATCH_SIZE=5
EPOCHS=2
LEARNING_RATE=5.6e-5
WEIGHT_DECAY=0.01
LOG_EVERY=50
EVAL_EVERY = 500
LR_SCHEDULAR_TYPE = "linear"
TEMPERATURE=0.6
NUM_BEAM = 4
MAX_GEN_LEN = 30

PATH_TO_MODEL="/content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000"

In [49]:
import pandas as pd
from rouge_score import rouge_scorer
import numpy as np
from transformers import AutoModelForSeq2SeqLM,T5TokenizerFast
from transformers import AutoConfig

submission = pd.read_csv("/content/drive/MyDrive/ML Assignment/train.csv")
model = AutoModelForSeq2SeqLM.from_pretrained(PATH_TO_MODEL)
tokenizer = T5TokenizerFast.from_pretrained(PATH_TO_MODEL)
config = AutoConfig.from_pretrained(PATH_TO_MODEL)
submission=submission.loc[:20,:]

def predict(texts):
    # write code to output a list of title for each text input to the predict method
    corpus_data=texts.tolist()
    input_data = tokenizer(corpus_data, max_length=600, return_tensors='pt',truncation=True, padding=True,)
    title_idx = model.generate(
    input_data['input_ids'],
    temperature=TEMPERATURE,
    max_length=MAX_GEN_LEN,
    num_beams=NUM_BEAM,
    early_stopping=True)
    prediction=[]
    for idx in range(len(corpus_data)):
      title = tokenizer.decode(title_idx[idx], skip_special_tokens=True, clean_up_tokenization_spaces=False,)
      prediction.append(title)
    score=evaluate(prediction,submission['title'].tolist())
    print(score)
    return prediction

def test_model():
    pred = predict(submission['text'])
    submission['predicted_title'] = pred
    submission.to_csv('submission.csv',index=False)


def evaluate(model_output,actual_titles):
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scores = list()
    for output,actual in zip(model_output,actual_titles):
        s = scorer.score(output,actual)
        scores.append(s['rouge1'].fmeasure)

    print('Evaluation result',np.mean(scores))
    return scores




if __name__=="__main__":
    #write model loading code here
    test_model()


loading configuration file /content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000/config.json
Model config T5Config {
  "_name_or_path": "/content/drive/MyDrive/ML Assignment/model-t5-base/checkpoint-5000",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3

Evaluation result 0.490750262252932
[0.5185185185185186, 0.6666666666666666, 0.42857142857142855, 0.3846153846153846, 0.4545454545454546, 0.28571428571428564, 0.1, 0.3, 0.43478260869565216, 0.5263157894736842, 0.28571428571428564, 0.631578947368421, 0.4615384615384615, 0.7499999999999999, 0.7, 0.5454545454545454, 0.5, 0.7500000000000001, 0.5, 0.5217391304347826, 0.56]
Evaluation result 0.490750262252932
[0.5185185185185186, 0.6666666666666666, 0.42857142857142855, 0.3846153846153846, 0.4545454545454546, 0.28571428571428564, 0.1, 0.3, 0.43478260869565216, 0.5263157894736842, 0.28571428571428564, 0.631578947368421, 0.4615384615384615, 0.7499999999999999, 0.7, 0.5454545454545454, 0.5, 0.7500000000000001, 0.5, 0.5217391304347826, 0.56]
